In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Aug 15 13:01:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np
import pandas as pd 
import numpy as np
from sklearn import tree
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn import  naive_bayes
from math import sqrt 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn import linear_model, model_selection

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/train_data.csv')
df_test=pd.read_csv('/content/drive/MyDrive/test_data.csv')

In [5]:
print('training datasets : ',df['Label'].value_counts())
print('----------------------------------------')
print('testing datasets : ',df_test['Label'].value_counts()) 

training datasets :  1    215487
0     34043
Name: Label, dtype: int64
----------------------------------------
testing datasets :  1    140000
0     56965
Name: Label, dtype: int64


In [6]:
np.isinf(df).values.sum()
df.replace([np.inf, -np.inf], np.nan, inplace=True)
np.isinf(df).values.sum()
df.dropna(axis=0,inplace=True)
df.isnull().sum()
np.isinf(df_test).values.sum()
df_test.replace([np.inf, -np.inf], np.nan, inplace=True)
np.isinf(df_test).values.sum()
df_test.dropna(axis=0,inplace=True)
df_test.isnull().sum()

Flow Duration                  0
Total Fwd Packets              0
Total Backward Packets         0
Total Length of Fwd Packets    0
Total Length of Bwd Packets    0
Fwd Packet Length Max          0
Fwd Packet Length Min          0
Fwd Packet Length Mean         0
Fwd Packet Length Std          0
Bwd Packet Length Max          0
Bwd Packet Length Min          0
Bwd Packet Length Mean         0
Bwd Packet Length Std          0
Flow Bytes/s                   0
Flow Packets/s                 0
Flow IAT Mean                  0
Flow IAT Std                   0
Flow IAT Max                   0
Flow IAT Min                   0
Fwd IAT Total                  0
Fwd IAT Mean                   0
Fwd IAT Std                    0
Fwd IAT Max                    0
Fwd IAT Min                    0
Bwd IAT Total                  0
Bwd IAT Mean                   0
Bwd IAT Std                    0
Bwd IAT Max                    0
Bwd IAT Min                    0
Fwd PSH Flags                  0
Bwd Header

In [7]:
x = df.iloc[:, : -1] # Take the features, except the last column, which is the target 
y = df.iloc[:, -1]
df_test_target=df_test.iloc[:,-1]
df_test_train=df_test.iloc[:,:-1]

In [8]:
x_train, x_val, y_train, y_val =model_selection.train_test_split(x,y, test_size=0.2, random_state=2021,shuffle=True)

logistic regression


In [9]:
paramater_lr={'fit_intercept':[False,True],
           'C':range(1,30,2),
           'penalty':['l2','l1'],
           'random_state':[2021]}

In [10]:
lr_cv=GridSearchCV(linear_model.LogisticRegression(),paramater_lr,refit=True,verbose = 3,n_jobs=-1,cv=5)
lr_cv.fit(x_train, y_train)
print('Best parameter for fare_amount in RandomForestRegressor: ',lr_cv.best_params_)


Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   52.7s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  8.9min finished


Best parameter for fare_amount in RandomForestRegressor:  {'C': 25, 'fit_intercept': True, 'penalty': 'l2', 'random_state': 2021}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [11]:
predict_val=lr_cv.predict(x_val)
predict_test=lr_cv.predict(df_test_train)
print('precision and recall for validation data:', metrics.precision_score(y_val, predict_val), metrics.recall_score(y_val,predict_val))
print('f1 score for validation datasets: ',f1_score(y_val,predict_val))
print('----------------------------------------------------------------------------------')

print('precision and recall for validation data:', metrics.precision_score(df_test_target, predict_test), metrics.recall_score(df_test_target,predict_test))
print('f1 score for test datasets: ',f1_score(df_test_target,predict_test))


precision and recall for validation data: 0.949361777306656 0.9980498223945395
f1 score for validation datasets:  0.9730971648463584
----------------------------------------------------------------------------------
precision and recall for validation data: 0.851508358419266 0.9931126184002151
f1 score for test datasets:  0.9168752887950951


In [13]:
ovr_report=f'''
Confusion Matrix:
{metrics.confusion_matrix(df_test_target,predict_test)}
Accuracy: {round(metrics.accuracy_score(df_test_target,predict_test), 3)}
'''
print(ovr_report)
print('The classification report of binary:\n {}'
      .format(metrics.classification_report(df_test_target,predict_test)))
y_pred_prob_test = lr_cv.predict_proba(df_test_train)[:,1]
y_pred_prob_val= lr_cv.predict_proba(x_val)[:,1]
print('AUC_test:', metrics.roc_auc_score(df_test_target, y_pred_prob_test))
print('AUC_val:', metrics.roc_auc_score(y_val, y_pred_prob_val))


Confusion Matrix:
[[ 33122  23184]
 [   922 132946]]
Accuracy: 0.873

The classification report of binary:
               precision    recall  f1-score   support

           0       0.97      0.59      0.73     56306
           1       0.85      0.99      0.92    133868

    accuracy                           0.87    190174
   macro avg       0.91      0.79      0.83    190174
weighted avg       0.89      0.87      0.86    190174

AUC_test: 0.9142506795804095
AUC_val: 0.9482073153159285


# Search Best important features

In [14]:
LR=linear_model.LogisticRegression(C= 15, fit_intercept=True, penalty='l2', random_state= 2021)

In [15]:
LR.fit(x_train, y_train)
importance = LR.coef_[0]

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [16]:

importance=pd.DataFrame(x_train.columns)
importance['score']=LR.coef_[0]
importance.sort_values(by='score',ascending=True)

,0,score
17,Flow IAT Max,-9.795005e-07
0,Flow Duration,-7.718631e-07
37,Packet Length Variance,-6.457643e-07
58,Idle Min,-4.416517e-07
55,Idle Mean,-3.821069e-07
53,Active Max,-3.652178e-07
51,Active Mean,-3.151125e-07
24,Bwd IAT Total,-2.835426e-07
54,Active Min,-2.801605e-07
18,Flow IAT Min,-2.372410e-07
